# OpenAPI를 이용한 논문, 기사, 글 요약 및 번역

https://jehyunlee.github.io/2022/07/02/Python-DS-106-aaicon/

### Reference

```
이제현, 유시현, 김창기, 김현구, "Open API를 활용한 고속 논문 분석",
실용인공지능학회지 vol.1 p.9, 2022
```

## API 키 발급

- RapidAPI : https://rapidapi.com/developer/new
- 네이버 : https://developers.naver.com/apps/#/register?api=ppg_n2mt

In [1]:
rapidapi_key = 'ac7a118e41msh6fa77e673997426p1860c2jsn130536d944bf'
naver_client_id = 'JnlJAvl5Qc2Hud9taeOP'
naver_client_secret = 'DFzwkcft1d'
#파파고 1일 10,000글자 무료
#rapid --> 1개월 100개 무료

# TLDRThis

- https://rapidapi.com/tldrthishq-tldrthishq-default/api/tldrthis/

---

## 요약의 종류

1. Abstractive(Human-like) summarization

> Abstractive summarization(생성 요약)은 기존 Input text를 그대로 인용하지 않고, 기존의 내용을 새롭게 re-phrasing 하여 Summary를 생성하는 요약 모델입니다.

2. Extractive summarization

> 반면에 Extractive summarization(추출 요약)은 기존 Input text에 존재하는 중요한 단어를 그대로 사용하여 Summary를 생성하는 요약 모델입니다.

출처 : https://supkoon.tistory.com/40

In [2]:
import requests
from pprint import pprint

### 예시 논문

Attention Is All You Need
- 페이지 : https://arxiv.org/abs/1706.03762
- 본문(pdf) : https://arxiv.org/pdf/1706.03762.pdf

## 요약

In [7]:
url = "https://tldrthis.p.rapidapi.com/v1/model/abstractive/summarize-url/"

payload = {
    "url": "test.pdf", # 주소
    "min_length": 100, # 최소 길이
    "max_length": 300, # 최대 길이
    "is_detailed": False # 한 문장으로 반환할 것인지 여부
}

headers = {
    "content-type": "application/json",
    "X-RapidAPI-Key": rapidapi_key,
    "X-RapidAPI-Host": "tldrthis.p.rapidapi.com"
}

response = requests.request("POST", url, json=payload, headers=headers)

pprint(response.json())

{'detail': [{'loc': ['body', 'url'],
             'msg': 'invalid or missing URL scheme',
             'type': 'value_error.url.scheme'}]}


In [5]:
summary = response.json()['summary'][0].strip()

print(summary)

KeyError: 'summary'

## 번역

네이버 파파고 API

In [31]:
url = "https://openapi.naver.com/v1/papago/n2mt"

payload = {
    "source": "en",
    "target": "ko",
    "text": summary,
}

headers = {
    "content-type": "application/json",
    "X-Naver-Client-Id": naver_client_id,
    "X-Naver-Client-Secret": naver_client_secret,
}

response = requests.request("POST", url, json=payload, headers=headers)

pprint(response.json())

{'message': {'@service': 'naverservice.nmt.proxy',
             '@type': 'response',
             '@version': '1.0.0',
             'result': {'dict': None,
                        'engineType': 'UNDEF_MULTI_SENTENCE',
                        'pivot': None,
                        'srcLangType': 'en',
                        'tarDict': None,
                        'tarLangType': 'ko',
                        'translatedText': '우리는 주의 메커니즘만을 기반으로 하는 새로운 간단한 네트워크 '
                                          '아키텍처인 트랜스포머를 제안한다. 두 가지 기계 번역 작업에 '
                                          '대한 실험은 이러한 모델이 품질 면에서 우수하면서도 병렬화가 더 '
                                          '가능하고 교육하는 데 훨씬 적은 시간이 필요하다는 것을 '
                                          '보여준다. Transformer는 훨씬 더 많은 병렬화를 '
                                          '가능하게 하며 8개의 P100 GPU에서 12시간 정도 교육을 '
                                          '받은 후 번역 품질에서 새로운 최첨단 기술에 도달할 수 '
                                          '있습니다. 순차 연산을 줄이는 목표

In [32]:
print(response.json()['message']['result']['translatedText'])

우리는 주의 메커니즘만을 기반으로 하는 새로운 간단한 네트워크 아키텍처인 트랜스포머를 제안한다. 두 가지 기계 번역 작업에 대한 실험은 이러한 모델이 품질 면에서 우수하면서도 병렬화가 더 가능하고 교육하는 데 훨씬 적은 시간이 필요하다는 것을 보여준다. Transformer는 훨씬 더 많은 병렬화를 가능하게 하며 8개의 P100 GPU에서 12시간 정도 교육을 받은 후 번역 품질에서 새로운 최첨단 기술에 도달할 수 있습니다. 순차 연산을 줄이는 목표는 확장 신경 GPU[16], ByteNet[18] 및 ConvS2S의 기초를 형성한다.


## 함수화

In [48]:
def summarize_and_translate(article_url, min_length=100, max_length=300):
    url = "https://tldrthis.p.rapidapi.com/v1/model/abstractive/summarize-url/"

    payload = {
        "url": article_url, # 주소
        "min_length": min_length, # 최소 길이
        "max_length": max_length, # 최대 길이
        "is_detailed": False # 한 문장으로 반환할 것인지 여부
    }

    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": rapidapi_key,
        "X-RapidAPI-Host": "tldrthis.p.rapidapi.com"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    summary = response.json()['summary'][0].strip()
    
    url = "https://openapi.naver.com/v1/papago/n2mt"

    payload = {
        "source": "en",
        "target": "ko",
        "text": summary,
    }

    headers = {
        "content-type": "application/json",
        "X-Naver-Client-Id": naver_client_id,
        "X-Naver-Client-Secret": naver_client_secret
    }

    response = requests.request("POST", url, json=payload, headers=headers)
    
    return response.json()['message']['result']['translatedText']

In [50]:
summarize_and_translate("https://arxiv.org/pdf/1706.03762.pdf", 50, 100)

'우리는 주의 메커니즘만을 기반으로 하는 새로운 간단한 네트워크 아키텍처인 트랜스포머를 제안한다. 두 가지 기계 번역 작업에 대한 실험은 이러한 모델이 품질 면에서 우수하면서도 병렬화가 더 가능하고 교육하는 데 훨씬 적은 시간이 필요하다는 것을 보여준다. 트랜스포머는 훨씬 더 많은 병렬화를 가능하게 하며 8개의 P100에서 12시간 동안만 교육을 받은 후 번역 품질에서 새로운 기술에 도달할 수 있습니다.'

In [51]:
summarize_and_translate("https://arxiv.org/pdf/1905.11946.pdf", 100, 200)

'EfficientNet-B7은 ImageNet에서 최고 84.3%의 정확도를 달성하는 동시에 기존 최고의 ConvNet보다 8.4배 작고 추론 속도가 6.1배 빠르다. 우리의 EfficientNets는 다른 ConvNets를 크게 능가한다. 이전 작업에서는 깊이, 너비, 이미지 크기 등 세 가지 차원 중 하나만 스케일링하는 것이 일반적이다. 단순하지만 매우 효과적인 복합 계수를 사용하여 깊이/폭/해상도의 모든 차원을 균일하게 확장하는 단순하지만 효과적인 복합 스케일링 방법을 제안한다. 예를 들어, 우리가 2N배 더 많은 계산 자원을 사용하고 싶다면, 우리는 단순히 원래 모델에서 네트워크를 작은 그리드만큼 늘릴 수 있다.'